In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce

In [ ]:
# dodac produkcje zwracaja nowe pod grafy <check> 
# dodac do node'ow typu I id parenta, pierwszy node Typu I bedzie zawierac parenta -1 <check>
# layer ma byc lista <check>
# wizualizacja warstwy (brutem) <check>
# dodac check czy mozna wywolac <check>
#czyscic kernel i out przed pushem

In [ ]:
class Id_creator:
    def __init__(self):
        self.last_id = -1
    def get_id(self):
        self.last_id +=1
        return self.last_id
    def __call__(self):
        return self.get_id()
    
    
class CannotExecuteProduction(Exception):
        pass
    
    
class Graph_layers:
    def __init__(self):
        self._node_id_gen =  Id_creator()
        G = nx.Graph()
        G.add_nodes_from([(self._node_id_gen(), {'pos': (0,0),'type': 'e'})])
        self._layers=[[G]]
        
    def get_layer(self, i):
        return self._layers[i]
    
    def add_to_layer(self,i,G):
        return self._layers[i].append(G)
    
    def add_to_last_layer(self,G):
        return self._layers[-1].append(G)
        
    def add_new_layer(self, G_new):
        self._layers.append([G_new])            
        
    def get_last_layer_index(self):
        return len(self._layers) - 1
    
    def get_last_layer(self,i):
        return self._layers[i]
    
    def display_i_layer(self, i):
        G_layer = self._layers[i]
        G = self._layers[i][0] if  len(self._layers[i])==1 else reduce(nx.algorithms.operators.binary.compose,G_layer)
        
        pos = nx.get_node_attributes(G, 'pos')
        nx.draw_networkx(G, pos)
        
    def get_node_id_gen(self):
        return self._node_id_gen

In [ ]:
def p1(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
    assert(G.nodes[base_node_id]['type'].lower() == 'e' )
    assert(len(G.nodes)==1)
    all_new_nodes = []
    all_new_edges = []
    base_pos = G.nodes[base_node_id]['pos']
    x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    i_node_x, i_node_y = base_pos[0], base_pos[1]
    i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
    all_new_nodes.append(i_node)
    
    half_side_len = side_len/2
    e_nodes = [
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'})
    ]
    for i in range(len(e_nodes)):
        all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
        all_new_edges.extend([(i_node[0],e_nodes[i][0])])
    
    all_new_nodes.extend(e_nodes)
    nG = nx.Graph()
    nG.add_nodes_from(all_new_nodes)
    nG.add_edges_from(all_new_edges)
    return nG, i_node


In [ ]:
def conc_duplicates(G):
    org_nodes = list(G.nodes(data=True))

    org_nodes_dict = {}
    for node_id,data in org_nodes:
        try:
            org_nodes_dict[data['pos']].append(node_id)
        except KeyError:
            org_nodes_dict[data['pos']]=[node_id]

    nodes_to_replace = {} #keys nodes to be deleted, values nodes to replace them
    for key,ids in org_nodes_dict.items():
        if len(ids) > 1:
            for id_ in ids[1:]:
                nodes_to_replace[id_] = ids[0]
                
    for old_node, new_node in nodes_to_replace.items():
        neighbors = list(G.neighbors(old_node))
        for n in neighbors:
            G.remove_edge(old_node,n)
            G.add_edge(new_node,n)
        G.remove_node(old_node)
    return G

In [ ]:
def avg_pos(pos1, pos2):
    return ((pos1[0] + pos2[0])/2, (pos1[1] + pos2[1])/2)

def rm_edge_if_exists(G,n1id,n2id):
    try:
        G.remove_edge(n1id,n2id)
    except nx.NetworkXError:
        pass
    


#     e0 - - - e1
#      | \    / |
#      |  I0    |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0 - - - n0 - - - e1
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    n1 - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2 - - - n4 - - - e3

def p2(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    if len(es) != 4:
        raise CannotExecuteProduction
    e0, e1, e2, e3 = None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    if not(G.has_edge(e0[0],e1[0]) and\
            G.has_edge(e1[0],e3[0]) and\
            G.has_edge(e3[0],e2[0]) and\
            G.has_edge(e0[0],e2[0]) and\
            G.has_edge(e0[0],base_node_id) and\
            G.has_edge(e1[0],base_node_id) and\
            G.has_edge(e2[0],base_node_id) and\
            G.has_edge(e3[0],base_node_id)\
            ):
        raise CannotExecuteProduction
    #prepare all new verticies accord to map above
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG
#     conc_duplicates(G)


In [ ]:
graph_layers = Graph_layers()
graph_layers.display_i_layer(0)

In [ ]:
base_node = list(graph_layers.get_layer(0)[0].nodes(data=True))[0]
#indexing from 0 
first_layer_G = graph_layers.get_layer(0)[0].copy()

#apply p1 production
G, i_node = p1(first_layer_G, base_node[0], graph_layers.get_node_id_gen()) 
graph_layers.add_new_layer(G)

graph_layers.display_i_layer(1)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(1)[0]
nG = p2(G, 1, graph_layers.get_node_id_gen())
graph_layers.add_new_layer(nG)
graph_layers._layers[2][0].nodes(data=True)
graph_layers.display_i_layer(2)

In [ ]:
#apply p2 production 4 times
G = graph_layers.get_layer(2)[0]
graph_layers.add_new_layer( p2(G, 15, graph_layers.get_node_id_gen()))
graph_layers.add_to_last_layer(p2(G, 18, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(2)[0]
graph_layers.add_to_last_layer(p2(G, 17, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(2)[0]
graph_layers.add_to_last_layer(p2(G, 16, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#     e0 - m0 - e1
#      | \    / |
#     m1   I0   m2 
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0 - - - m0 - - - e1
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    m1 - - - n0 - - - m2
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2 - - - n1 - - - e3
def p5(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    
    # check if I has 4 neighbours
    if len(es) != 4:
        raise CannotExecuteProduction
    
    # assign e
    e0, e1, e2, e3 = None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
        
    # check if each e has 3 neighbours
    e0_neigh = list(G.neighbors(e0[0]))
    e1_neigh = list(G.neighbors(e1[0]))
    e2_neigh = list(G.neighbors(e2[0]))
    e3_neigh = list(G.neighbors(e3[0]))
    if (len(e0_neigh) != 3 or\
       len(e1_neigh) != 3 or\
       len(e2_neigh) != 3 or\
       len(e3_neigh) != 3):
        raise CannotExecuteProduction
    
    # assign m & check m (x,y)
    m0, m1, m2 = None, None, None
    e0X, e0Y = e0[1]['pos']
    e1X, e1Y = e1[1]['pos']
    e2X, e2Y = e2[1]['pos']
    e3X, e3Y = e3[1]['pos']
    for node_x in e0_neigh:
        x,y = G.nodes[node_x]['pos']
        if x==(e0X+e1X)/2 and y==(e0Y+e1Y)/2:
            m0 = (node_x,G.nodes[node_x])
        if x==(e0X+e2X)/2 and y==(e0Y+e2Y)/2:
            m1 = (node_x,G.nodes[node_x])
    for node_x in e1_neigh:
        x,y = G.nodes[node_x]['pos']
        if x==(e1X+e3X)/2 and y==(e1Y+e3Y)/2:
            m2 = (node_x,G.nodes[node_x])
    if m0 is None or m1 is None or m2 is None:
        raise CannotExecuteProduction
    
    # check if each m has 2 neighbours
    m0_neigh = list(G.neighbors(m0[0]))
    m1_neigh = list(G.neighbors(m1[0]))
    m2_neigh = list(G.neighbors(m2[0]))
    if (len(m0_neigh) != 2 or\
       len(m1_neigh) != 2 or\
       len(m2_neigh) != 2):
        raise CannotExecuteProduction
    
    # check I0 edges
    if not(G.has_edge(e0[0],base_node_id) and\
            G.has_edge(e1[0],base_node_id) and\
            G.has_edge(e2[0],base_node_id) and\
            G.has_edge(e3[0],base_node_id)\
            ):
        raise CannotExecuteProduction
        
    # check each e and m edges
    if not(G.has_edge(e0[0], m0[0]) and\
           G.has_edge(e0[0], m1[0]) and\
           G.has_edge(e1[0], m0[0]) and\
           G.has_edge(e1[0], m2[0]) and\
           G.has_edge(e2[0], m1[0]) and\
           G.has_edge(e2[0], e3[0]) and\
           G.has_edge(e3[0], m2[0])\
            ):
        raise CannotExecuteProduction
        
    #prepare all new verticies accord to map above
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    m0 = (n_id_gen(),{'pos' : m0[1]['pos'],'type': m0[1]['type']})
    m1 = (n_id_gen(),{'pos' : m1[1]['pos'],'type': m1[1]['type']})
    m2 = (n_id_gen(),{'pos' : m2[1]['pos'],'type': m2[1]['type']})
    
    n0 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    
    # add all new edges
    new_edges = [
        (e0[0],m0[0]),(m0[0],e1[0]),
        (e0[0],I1[0]),(m0[0],I1[0]), (m0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],m1[0]),(m0[0],n0[0]),(e1[0],m2[0]),
        (m1[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(m2[0],I2[0]),
        (m1[0],n0[0]),(n0[0],m2[0]),
        (m1[0],I3[0]),(n0[0],I3[0]), (n0[0],I4[0]),(m2[0],I4[0]),
        (m1[0],e2[0]),(n0[0],n1[0]),(m2[0],e3[0]),
        (e2[0],I3[0]),(n1[0],I3[0]), (n1[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n1[0]),(n1[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,m0,m1,m2,n0,n1,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG

In [ ]:
graph_layers_p5 = Graph_layers()
base_node_p5 = list(graph_layers_p5.get_layer(0)[0].nodes(data=True))[0]
#indexing from 0 
first_layer_G_p5 = graph_layers_p5.get_layer(0)[0].copy()

#apply p1 production
G_p5, i_node = p1(first_layer_G_p5, base_node_p5[0], graph_layers_p5.get_node_id_gen()) 

#temper the graph
rm_edge_if_exists(G_p5, 2, 3)
rm_edge_if_exists(G_p5, 2, 5)
rm_edge_if_exists(G_p5, 3, 4)
e0 = (2, G_p5.nodes[2])
e1 = (3, G_p5.nodes[3])
e2 = (5, G_p5.nodes[5])
e3 = (4, G_p5.nodes[4])
m0 = (111,{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type': 'e'})
m1 = (222,{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type': 'e'})
m2 = (333,{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type': 'e'})
new_edges = [
     (e0[0], m1[0]), (m1[0], e2[0]),
     (e0[0], m0[0]), (m0[0], e1[0]),
     (e1[0], m2[0]), (m2[0], e3[0]),
    ]
G_p5.add_nodes_from([m0,m1,m2])
G_p5.add_edges_from(new_edges)
graph_layers_p5.add_new_layer(G_p5)
graph_layers_p5.display_i_layer(1)

In [ ]:
#apply p5 production
G_p5 = graph_layers_p5.get_layer(1)[0]
nG_p5 = p5(G_p5, 1, graph_layers_p5.get_node_id_gen())
graph_layers_p5.add_new_layer(nG)
graph_layers_p5._layers[2][0].nodes(data=True)
graph_layers_p5.display_i_layer(2)

In [ ]:
# p5 tests
import unittest
from collections import defaultdict

def run_test_p5():
    idc = Id_creator()
    
    # do głupiego debugowania
    def draw(g):
        pos = nx.get_node_attributes(g, 'pos')
        nx.draw_networkx(g, pos)
    
    # funkcje pomocniczne, klasa z testami poniżej
    # returns Graph, I id, list of e ids
    def make_base_graph():
        ids = [idc() for x in range(5)]
        nodes = [[i, {'type': 'e'}] for i in ids]
        i = nodes[0]
        es = nodes[1:]
        i[1]['type'] = 'i'
        
        edges = []
        pairs = [[0,1],[0,2],[1,3],[2,3]]

        for e in es:
            edges.append((i[0], e[0]))

        for a,b in pairs:
            edges.append((es[a][0], es[b][0]))
        
        i[1]['pos'] = (0,0)
        poses = [(-1, 1), (1, 1), (-1, -1), (1, -1)]
        for idx, pos in enumerate(poses):
            es[idx][1]['pos'] = pos
        
        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)

        return g, ids[0], ids[1:]
    
    def make_basic_graph(edges=[[0,1], [0,2], [1,3]]):
        g, i, es = make_base_graph()
        for ai, bi in edges:
            a, b = es[ai], es[bi]
            add_m_between(g, a, b)
        return (g, i, es)
    
    def add_m_between(g, ia, ib):
        m = idc()
        a = g.nodes[ia]['pos']
        b = g.nodes[ib]['pos']
        g.add_nodes_from([(m, {'type': 'e', 'pos': avg_pos(a,b)})])
        g.add_edges_from([[ia,m], [m,ib]])
        g.remove_edge(ia, ib)
        return m


    class TestProduction5(unittest.TestCase):
        def is_result_ok(self, g, base_node_id):
            def where_nodes(attr, t, nodes=None):
                ns = g.nodes if nodes == None else nodes
                attrs = nx.get_node_attributes(g, attr)
                return [n for n in ns if attrs[n] == t]

            def find_nodes_between(nodes):
                # for a given set of nodes [e0, e1, e2, ...]
                # find nodes (of type 'e') that are neighbors of exactly two of these nodes
                m_scores = defaultdict(int)
                for e in nodes:
                    for neighbor in g.neighbors(e):
                        if g.nodes[neighbor]['type'] == 'e':
                            m_scores[neighbor] += 1
                ms = [k for k, v in m_scores.items() if v == 2]
                return ms

            def is_node_between(m, a, b):
                expected_pos = avg_pos(g.nodes[a]['pos'], g.nodes[b]['pos'])
                return expected_pos == g.nodes[m]['pos']
    
            Is = where_nodes('parent', base_node_id, nodes=where_nodes('type', 'I'))

            # dla każdej ćwiartki
            for I in Is:
                es = list(g.neighbors(I))
                if len(es) != 4:
                    return False
                posc = lambda c: sum(g.nodes[n]['pos'][c] for n in es)/4
                # czy I znajduje się w średniej współrzędnych wszystkich sąsiadów
                self.assertEqual(g.nodes[I]['pos'], (posc(0), posc(1)))

            # czy pary I mają jednego/dwóch wspólnych sąsiadów
            for a, b in [[0,1], [0,2], [2,3], [1,3]]:
                I1, I2 = Is[a], Is[b]
                self.assertEqual(2, len(find_nodes_between([I1, I2])))
            for a, b in [[0,3], [1,2]]:
                I1, I2 = Is[a], Is[b]
                between = find_nodes_between([I1, I2])
                self.assertEqual(1, len(between))
                self.assertTrue(is_node_between(between[0], I1, I2))
            
            # czy I razem mają 9 e
            eses = [list(g.neighbors(I)) for I in Is]
            # flatten
            eses = [item for items in eses for item in items]
            eses = list(set(eses))
            self.assertEqual(len(eses), 9)
        

        def test_super_basic(self):
            # najprostszy test case
            g, i, es = make_basic_graph()
            p5_result = p5(g, i, idc)
            self.is_result_ok(p5_result, i)
            
            # odnajdź node po pozycji + asercja że istnieje dokładnie jeden
            def find_node(graph, pos):
                attrs = nx.get_node_attributes(graph, 'pos')
                results = [k for k, v in attrs.items() if v == pos]
                self.assertEqual(len(results), 1)
                return results[0]

            # sprawdzenie kilku nodów (górnych) i porównanie do lewej strony
            poses = nx.get_node_attributes(p5_result, 'pos')
            cord = lambda n: list(map(lambda pos: pos[n], poses.values()))
            topleft = find_node(p5_result, (min(cord(0)), max(cord(1))))
            topright = find_node(p5_result, (max(cord(0)), max(cord(1))))
            self.assertFalse(topleft in p5_result.neighbors(topright))
            
            # górny środkowy:
            topmid = find_node(p5_result, avg_pos(p5_result.nodes[topleft]['pos'], p5_result.nodes[topright]['pos']))
            self.assertTrue(topmid in p5_result.neighbors(topleft))
            self.assertTrue(topmid in p5_result.neighbors(topright))
            
            # czy istnieją odpowidniki po lewej stronie produkcji:
            
            org_topleft = find_node(g, p5_result.nodes[topleft]['pos'])
            org_topright = find_node(g, p5_result.nodes[topright]['pos'])
            org_topright = find_node(g, p5_result.nodes[topmid]['pos'])

        def test_subgraph_passes(self):
            # czy aplikuje dla podgrafu
            # dodaję ekstra wierzchołek i krawędź w kilku miejscach
            def add_to(g, n):
                some_e_id = es[ei]
                new_id = idc()
                g.add_nodes_from([(new_id, {'type': 'e', 'pos': (0, 0.5)})])
                g.add_edge(new_id, some_e_id)

            for ei in range(4):
                g, i, es = make_base_graph()
    
                for ai, bi in [[0,1], [0,2], [1,3]]:
                    a, b = es[ai], es[bi]
                    add_m_between(g, a, b)
                add_to(g, es[ei])
                self.is_result_ok(p5(g, i, idc))

            for m in range(3):
                g, i, es = make_base_graph()
                ms = []
                for ai, bi in [[0,1], [0,2], [1,3]]:
                    a, b = es[ai], es[bi]
                    ms.append(add_m_between(g, a, b))
                add_to(g, ms[m])
                self.is_result_ok(p5(g, i, idc))
            # kilka ekstra wierzchołków
            g, i, es = make_base_graph()
            for e in es:
                add_to(g, e)
            self.is_result_ok(p5(g, i, idc))


        def test_sides(self):
            g, i, es = make_basic_graph(edges=[[0,1], [0,2], [2,3]])
            self.is_result_ok(p5(g, i, idc), i)
            g, i, es = make_basic_graph(edges=[[0,1], [1,3], [2,3]])
            self.is_result_ok(p5(g, i, idc), i)

        def test_upsidedown(self):
            g, i, es = make_basic_graph(edges=[[0,2], [2,3], [1,3]])
            self.is_result_ok(p5(g, i, idc), i)

        # testy sprawdzające czy produkcja się nie wykona kiedy nie powinna:
        
        def test_complete(self):
            # przypadek dla grafu do lewej strony producji 6
            g, i, es = make_basic_graph(edges=[[0,1], [0,2], [2,3], [1,3]])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
                
        def test_mislabel(self):
            # na obecnosc etykiety type: e
            g, i, es = make_base_graph()
            for ei in range(4):
                e = es[ei]
                g.nodes[e]['type'] = 'qweqweqwe'
                with self.assertRaises(CannotExecuteProduction):
                    p5(g, i, idc)

        def test_missing_e(self):
            # po usunieciu node
            g, i, es = make_base_graph()
            for ei in range(4):
                g.remove_node(es[ei])
                with self.assertRaises(CannotExecuteProduction):
                    p5(g, i, idc)


        def test_missing_m(self):
            # przypadek dobry dla produkcji 4
            g, i, es = make_basic_graph(edges=[[0,2], [2,3]])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)

        def test_missing_edges(self):
            # brak krawędzi między I a e
            g, i, es = make_basic_graph()
            g.remove_edge(i, es[0])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
    
            # brak krawędzi między e a e
            g, i, es = make_basic_graph()
            g.remove_edge(es[2], es[3])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
                
        def test_wrong_coords_e(self):
            g, i, es = make_basic_graph()
            g.nodes[es[0]]['pos'] = (10.0, 20.0)
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
                
        def test_wrong_coords_i(self):
            g, i, es = make_basic_graph()
            g.nodes[i]['pos'] = (10.0, 20.0)
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)

    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestProduction5)
    unittest.TextTestRunner().run(suite)
run_test_p5()